```
Notebook: prepare_pl_seed_data.ipynb
Purpose: Transform the Sustainable Procurement Practices Excel file
         into normalized CSVs ready for seeding into the PL database.
```

In [1]:
pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


# 1. Configuration

In [25]:
import pandas as pd
import os
from pathlib import Path
import re
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed

In [3]:
# Input and output setup
BASE_DIR = Path(".")
INPUT_FILE = BASE_DIR / "USE-THIS _Sustainable Procurement Practices Library_Updated Changes_October 2025_1 to 5 ranking.xlsx"
SHEET_NAME = "Procurement Practices"
OUTPUT_DIR = BASE_DIR / "output"
OUTPUT_DIR.mkdir(exist_ok=True)

In [4]:
print(f"📘 Loading file: {INPUT_FILE.name}")
print(f"📄 Sheet: {SHEET_NAME}")
print(f"📂 Output dir: {OUTPUT_DIR.resolve()}\n")

📘 Loading file: USE-THIS _Sustainable Procurement Practices Library_Updated Changes_October 2025_1 to 5 ranking.xlsx
📄 Sheet: Procurement Practices
📂 Output dir: /Users/galihpratama/Sites/IDH-IDC/backend/source/transformer/procurement_library_v2_oct2025/output



# 2. Load the Excel sheet (with multi-row header)

In [5]:
print("🔹 Loading Excel sheet with multi-row header...")
df_raw = pd.read_excel(INPUT_FILE, sheet_name=SHEET_NAME, header=[0, 1])

🔹 Loading Excel sheet with multi-row header...


In [6]:
# Define groups we want to keep
GROUP_HEADERS = [
    "Mapping to Sourcing Strategy Cycle",
    "Mapping to Sustainable Procurement Principles",
    "Mapping to Value Chain Actors",
]

In [7]:
# Define the list of plain (non-grouped) columns that mark the start of practice data
PRACTICE_COLUMNS = [
    "Practice Intervention",
    "Intervention Definition",
    "Enabling Conditions",
    "Practical Application for Sustainable Procurement Cost, Revenue, Risk",
    "Farmer Rationale Income & Environement",
    "Risks & Trade Offs",
    "Intervention Impact Income",
    "Intervention Impact Environment",
    "Source / Evidence",
    "Implementation Time",
    "Implementation Cost / Effort",
    "Income Impact",
    "Environmental Impact",
]

In [8]:
def clean_header_text(s: str) -> str:
    """Normalize header text by removing newlines, double spaces, etc."""
    if pd.isna(s):
        return ""
    s = re.sub(r"[\r\n]+", " ", str(s))
    s = re.sub(r"\s+", " ", s).strip()
    return s

In [9]:
# Smarter flattening logic
clean_cols = []
current_group = None

for top, bottom in df_raw.columns:
    top, bottom = clean_header_text(top), clean_header_text(bottom)

    # Detect start of group
    if top in GROUP_HEADERS:
        current_group = top

    # If this column belongs to the plain practice section, stop grouping
    if bottom in PRACTICE_COLUMNS:
        current_group = None

    # Handle Unnamed (empty) top header
    if top.startswith("Unnamed") or top == "":
        if current_group:
            clean_cols.append(f"{current_group}|{bottom}")
        else:
            clean_cols.append(bottom)
    else:
        # Normal grouped column
        if current_group and (top in GROUP_HEADERS):
            clean_cols.append(f"{current_group}|{bottom}")
        else:
            clean_cols.append(bottom)

In [10]:
# Final cleanup pass
df_raw.columns = [c.strip().replace("  ", " ") for c in clean_cols]

In [11]:
# Drop any columns that start with 'Unnamed' (extra empty Excel columns)
df_raw = df_raw.loc[:, ~df_raw.columns.str.contains("^Unnamed")]

# Optional: trim spaces again just in case
df_raw.columns = df_raw.columns.str.strip()

In [12]:
print("✅ Final Cleaned Columns:")
print(df_raw.columns.tolist())
print(f"✅ Loaded {len(df_raw)} rows.\n")

display(df_raw.head(2))

✅ Final Cleaned Columns:
['Area', 'Mapping to Sourcing Strategy Cycle|Step 1 of Sourcing Strategy Cycle', 'Mapping to Sourcing Strategy Cycle|Step 2 of Sourcing Strategy Cycle2', 'Mapping to Sourcing Strategy Cycle|Step 3 of Sourcing Strategy Cycle3', 'Mapping to Sourcing Strategy Cycle|Step 4 of Sourcing Strategy Cycle4', 'Mapping to Sustainable Procurement Principles|1. Longer-term purchasing agreements', 'Mapping to Sustainable Procurement Principles|2. Improved payment terms and higher (farmgate) prices)', 'Mapping to Sustainable Procurement Principles|3. Develop and deepen equitable supply chain relationships', 'Mapping to Sustainable Procurement Principles|4. Improve efficiency and enhance transparency', 'Mapping to Sustainable Procurement Principles|5. Reduce volatility and risk for farmers', 'Mapping to Value Chain Actors|Farmer', 'Mapping to Value Chain Actors|Primary/ Secondary Processor', 'Mapping to Value Chain Actors|Trader', 'Mapping to Value Chain Actors|Manufacturer/ Pr

,Area,Mapping to Sourcing Strategy Cycle|Step 1 of Sourcing Strategy Cycle,Mapping to Sourcing Strategy Cycle|Step 2 of Sourcing Strategy Cycle2,Mapping to Sourcing Strategy Cycle|Step 3 of Sourcing Strategy Cycle3,Mapping to Sourcing Strategy Cycle|Step 4 of Sourcing Strategy Cycle4,Mapping to Sustainable Procurement Principles|1. Longer-term purchasing agreements,Mapping to Sustainable Procurement Principles|2. Improved payment terms and higher (farmgate) prices),Mapping to Sustainable Procurement Principles|3. Develop and deepen equitable supply chain relationships,Mapping to Sustainable Procurement Principles|4. Improve efficiency and enhance transparency,Mapping to Sustainable Procurement Principles|5. Reduce volatility and risk for farmers,...,"Practical Application for Sustainable Procurement Cost, Revenue, Risk",Farmer Rationale Income & Environement,Risks & Trade Offs,Intervention Impact Income,Intervention Impact Environment,Source / Evidence,Implementation Time,Implementation Cost / Effort,Income Impact,Environmental Impact
0,Environment,NaN,NaN,NaN,X,NaN,NaN,NaN,x,x,...,Agroecological Practices are most likely to em...,Environmental Perspective:\nSoil Health: A mix...,1. Buyer Perspective:\nRisks:\nSupply Chain Co...,Direct,Direct,1. https://environmentalevidencejournal.biomed...,5,3.0,5.0,5.0
1,Income,X,NaN,NaN,NaN,NaN,x,NaN,x,NaN,...,Corporate or brand sustainability commitments ...,Income Perspective\nMarket Access and Fair Pri...,1. Buyer Perspective:\n\nRisks:\nMisalignment ...,Indirect,Indirect,Based on old procurement framework of IDH and ...,1,1.0,5.0,5.0


# 3. Define category mappings

In [13]:
CATEGORY_MAP = {
    # Sourcing Strategy Cycle
    "Mapping to Sourcing Strategy Cycle|Step 1 of Sourcing Strategy Cycle": "Sourcing Strategy Cycle - Step 1",
    "Mapping to Sourcing Strategy Cycle|Step 2 of Sourcing Strategy Cycle2": "Sourcing Strategy Cycle - Step 2",
    "Mapping to Sourcing Strategy Cycle|Step 3 of Sourcing Strategy Cycle3": "Sourcing Strategy Cycle - Step 3",
    "Mapping to Sourcing Strategy Cycle|Step 4 of Sourcing Strategy Cycle4": "Sourcing Strategy Cycle - Step 4",

    # Sustainable Procurement Principles
    "Mapping to Sustainable Procurement Principles|1. Longer-term purchasing agreements": "Sustainable Procurement Principle - 1. Longer-term purchasing agreements",
    "Mapping to Sustainable Procurement Principles|2. Improved payment terms and higher (farmgate) prices)": "Sustainable Procurement Principle - 2. Improved payment terms and higher (farmgate) prices)",
    "Mapping to Sustainable Procurement Principles|3. Develop and deepen equitable supply chain relationships": "Sustainable Procurement Principle - 3. Develop and deepen equitable supply chain relationships",
    "Mapping to Sustainable Procurement Principles|4. Improve efficiency and enhance transparency": "Sustainable Procurement Principle - 4. Improve efficiency and enhance transparency",
    "Mapping to Sustainable Procurement Principles|5. Reduce volatility and risk for farmers": "Sustainable Procurement Principle - 5. Reduce volatility and risk for farmers",

    # Value Chain Actors
    "Mapping to Value Chain Actors|Farmer": "Value Chain Actor - Farmer",
    "Mapping to Value Chain Actors|Primary/ Secondary Processor": "Value Chain Actor - Primary/ Secondary Processor",
    "Mapping to Value Chain Actors|Trader": "Value Chain Actor - Trader",
    "Mapping to Value Chain Actors|Manufacturer/ Processor": "Value Chain Actor - Manufacturer/ Processor",
    "Mapping to Value Chain Actors|Retailer": "Value Chain Actor - Retailer",
}

# 4. Normalize: extract categories, attributes, and practices

In [14]:
print("🔹 Normalizing categories, attributes, and practices...")

🔹 Normalizing categories, attributes, and practices...


In [15]:
def clean_indicator_name(label: str) -> str:
    # Lowercase
    name = label.lower()
    # Replace special connectors
    name = name.replace("&", "_and_").replace("%", "_percent_")
    # Replace slashes and spaces
    name = re.sub(r"[\/\s]+", "_", name)
    # Remove punctuation and parentheses
    name = re.sub(r"[()\"',.:;!?]", "", name)
    # Collapse multiple underscores
    name = re.sub(r"_+", "_", name)
    # Trim trailing/leading underscores
    name = name.strip("_")
    return name

In [16]:
# Prepare lookup containers
categories, attributes, practices, practice_tags, indicators, scores = [], [], [], [], [], []

In [17]:
# Pre-create categories from CATEGORY_MAP (unique by prefix)
for cat_label in sorted(set(CATEGORY_MAP.values())):
    prefix = cat_label.split(" - ")[0].strip()
    if not any(c["name"] == prefix for c in categories):
        categories.append({"id": len(categories) + 1, "name": prefix, "description": None})

In [18]:
# Build attributes (linked to categories)
for full_label in sorted(CATEGORY_MAP.values()):
    prefix = full_label.split(" - ")[0].strip()
    cat_id = next(c["id"] for c in categories if c["name"] == prefix)
    attributes.append({
        "id": len(attributes) + 1,
        "category_id": cat_id,
        "label": full_label,
        "description": None,
    })

In [19]:
def text_to_html(text):
    """Convert structured text into readable HTML with headings, links, and paragraphs."""
    if pd.isna(text) or str(text).strip() == "":
        return ""
    
    lines = str(text).split("\n")
    formatted_lines = []

    for line in lines:
        line = line.strip()
        if not line:
            continue
        
        # Handle numbered/bullet lists
        if re.match(r"^\d+\.\s", line):
            if "http" in line:
                formatted_lines.append(f'<a href="{line[3:].strip()}" target="_blank">{line}</a><br/>')
            else:
                formatted_lines.append(f"<h3>{line}</h3>")

        # Convert titles to headings
        elif any(keyword.lower() in line.lower() for keyword in [
            "definition:", "enabling conditions", "business rationale",
            "farmer rationale", "risks:", "trade-offs:"
        ]):
            if len(line) < 90:
                formatted_lines.append(f"<h4>{line.replace(':', '')}</h4>")
            else:
                formatted_lines.append(f"<p>{line}</p>")

        # Convert URLs
        elif re.search(r"https?://\S+", line):
            line = re.sub(r"(https?://\S+)", r'<a href="\1" target="_blank">\1</a>', line)
            formatted_lines.append(f"<p>{line}</p>")

        # Bold label-value pairs
        elif ":" in line and not re.match(r"^\d+\.\s", line):
            parts = line.split(":", 1)
            if len(parts) == 2 and parts[1].strip():
                formatted_lines.append(f"<p><b>{parts[0]}:</b> {parts[1]}</p>")
            else:
                formatted_lines.append(f"<h3>{line}</h3>")
        else:
            formatted_lines.append(f"<p>{line}</p>")
    
    return "\n".join(formatted_lines)

In [20]:
# Build practice interventions
for _, row in df_raw.iterrows():
    label = str(row.get("Practice Intervention", "")).strip()
    if not label or label.lower() == "nan":
        continue

    practice = {
        "id": len(practices) + 1,
        "label": label,
        "intervention_definition": text_to_html(row.get("Intervention Definition")),
        "enabling_conditions": text_to_html(row.get("Enabling Conditions")),
        "business_rationale": text_to_html(row.get("Practical Application for Sustainable Procurement Cost, Revenue, Risk")),
        "farmer_rationale": text_to_html(row.get("Farmer Rationale Income & Environement")),
        "risks_n_trade_offs": text_to_html(row.get("Risks & Trade Offs")),
        "intervention_impact_income": text_to_html(row.get("Intervention Impact Income")),
        "intervention_impact_env": text_to_html(row.get("Intervention Impact Environment")),
        "source_or_evidence": text_to_html(row.get("Source / Evidence")),
    }
    practices.append(practice)

    # Handle tag links (X markers)
    for col, full_label in CATEGORY_MAP.items():
        val = str(row.get(col, "")).strip().lower()
        if val in ("x", "✓"):
            attr_id = next(a["id"] for a in attributes if a["label"] == full_label)
            practice_tags.append({
                "practice_intervention_id": practice["id"],
                "attribute_id": attr_id
            })

    # Handle indicator scores
    for indicator_name in [
        "Implementation Time",
        "Implementation Cost / Effort",
        "Income Impact",
        "Environmental Impact",
    ]:
        if pd.notna(row.get(indicator_name)):
            # Create indicator if not exists
            if not any(i["label"] == indicator_name for i in indicators):
                indicators.append({
                    "id": len(indicators) + 1,
                    "name": clean_indicator_name(indicator_name),
                    "label": indicator_name,
                    "description": None,
                })
            ind_id = next(i["id"] for i in indicators if i["label"] == indicator_name)
            scores.append({
                "id": len(scores) + 1,
                "practice_intervention_id": practice["id"],
                "indicator_id": ind_id,
                "score": float(row.get(indicator_name)),
            })

In [26]:
# -----------------------------------------------
# 1️⃣ Extract URLs from the source_or_evidence text
# -----------------------------------------------
def extract_urls_from_html(text):
    """Extract and clean URLs from text or HTML."""
    if pd.isna(text) or not text:
        return []
    urls = re.findall(r'(https?://\S+|www\.\S+|\S+\.(?:com|org|net))', str(text))
    clean_urls = []
    for u in urls:
        url = (
            u.replace("href=", "")
            .replace('"', "")
            .replace("'", "")
            .replace("(", "")
            .replace(")", "")
            .replace("</strong>", "")
            .split(">")[0]
        )
        if url.startswith("www."):
            url = f"http://{url}"
        clean_urls.append(url.strip())
    return clean_urls

In [27]:
# -----------------------------------------------
# 2️⃣ Validate URLs (HTTP HEAD -> fallback GET)
# -----------------------------------------------
def get_status_code(url, timeout=5):
    """Return HTTP status code or None if unreachable."""
    try:
        response = requests.head(url, allow_redirects=True, timeout=timeout)
        return int(response.status_code)
    except requests.RequestException:
        try:
            response = requests.get(url, allow_redirects=True, timeout=timeout)
            return int(response.status_code)
        except:
            return None

In [28]:
# -----------------------------------------------
# 3️⃣ Build dataframe from practices list
# -----------------------------------------------
df_practices = pd.DataFrame(practices)

# Extract URLs per practice
df_practices["source_urls"] = df_practices["source_or_evidence"].apply(extract_urls_from_html)

# Flatten all URLs for validation
urls_to_check = df_practices["source_urls"].explode().dropna().unique().tolist()

In [29]:
# -----------------------------------------------
# 4️⃣ Validate all unique URLs in parallel
# -----------------------------------------------
print(f"🔍 Validating {len(urls_to_check)} unique URLs...")

url_status_map = {}

with ThreadPoolExecutor(max_workers=10) as executor:
    future_to_url = {
        executor.submit(get_status_code, url): url for url in urls_to_check
    }
    for future in as_completed(future_to_url):
        url = future_to_url[future]
        url_status_map[url] = future.result()

🔍 Validating 91 unique URLs...


In [30]:
# -----------------------------------------------
# 5️⃣ Attach validation results to df_practices
# -----------------------------------------------
def map_status_list(url_list):
    if not isinstance(url_list, list):
        return []
    return [url_status_map.get(u, None) for u in url_list]

df_practices["source_urls_status"] = df_practices["source_urls"].apply(map_status_list)

In [32]:
# -----------------------------------------------
# 6️⃣ Create readable summary columns
# -----------------------------------------------
def summarize_valid_urls(urls, statuses):
    if not urls or not statuses:
        return ""
    return ", ".join(
        [f"{u} ({s})" for u, s in zip(urls, statuses) if s and s < 400]
    )

df_practices["valid_source_urls"] = df_practices.apply(
    lambda r: summarize_valid_urls(r["source_urls"], r["source_urls_status"]),
    axis=1
)

In [33]:
# -----------------------------------------------
# 7️⃣ Save back to CSV (includes all fields)
# -----------------------------------------------
output_file = "./output/practices_with_validated_sources.csv"
df_practices.to_csv(output_file, index=False)

print(f"✅ Validation complete — saved to {output_file}")
display(df_practices[["label", "source_or_evidence", "valid_source_urls"]].head(5))

✅ Validation complete — saved to ./output/practices_with_validated_sources.csv


,label,source_or_evidence,valid_source_urls
0,Agroecological Practices,"<a href=""https://environmentalevidencejournal....",https://environmentalevidencejournal.biomedcen...
1,Incorporate Business Strategy Needs,<p>Based on old procurement framework of IDH a...,
2,Buyer Sustainability Targets,<p>IDH Based Interviews (Frank Joosten)</p>\n<...,
3,Decarbonisation levers,"<a href=""https://www.greenbiz.com/article/why-...","https://www.greenbiz.com (200), https://www.gr..."
4,Comprehensive Value Chain Risk Assessment,"<a href=""https://consult.defra.gov.uk/water/ru...",https://consult.defra.gov.uk/water/rules-for-d...


In [34]:
print(f"✅ {len(categories)} categories")
print(f"✅ {len(attributes)} attributes")
print(f"✅ {len(practices)} practices")
print(f"✅ {len(practice_tags)} practice-attribute links")
print(f"✅ {len(indicators)} indicators")
print(f"✅ {len(scores)} indicator scores\n")

✅ 3 categories
✅ 14 attributes
✅ 27 practices
✅ 96 practice-attribute links
✅ 4 indicators
✅ 108 indicator scores



# 5. Save normalized data into CSV files

In [35]:
print("💾 Saving normalized CSVs to output/ ...")

💾 Saving normalized CSVs to output/ ...


In [36]:
def save_csv(data, name):
    df = pd.DataFrame(data)
    path = OUTPUT_DIR / f"{name}.csv"
    df.to_csv(path, index=False)
    print(f" - {name}.csv ({len(df)} rows)")
    return path

In [37]:
save_csv(categories, "pl_category")
save_csv(attributes, "pl_attribute")
save_csv(practices, "pl_practice_intervention")
save_csv(practice_tags, "pl_practice_intervention_tag")
save_csv(indicators, "pl_indicator")
save_csv(scores, "pl_practice_indicator_score")

print("\n✅ All CSVs successfully generated!")

 - pl_category.csv (3 rows)
 - pl_attribute.csv (14 rows)
 - pl_practice_intervention.csv (27 rows)
 - pl_practice_intervention_tag.csv (96 rows)
 - pl_indicator.csv (4 rows)
 - pl_practice_indicator_score.csv (108 rows)

✅ All CSVs successfully generated!
